In [3]:
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
#Import libraries:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional     scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search
import os 
os.chdir(r'C:\Users\txy9r\Desktop\ML\titanic')

C:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [10]:
data = pd.read_csv('dfdf.csv')
test_data = data[data['Survived'].isnull()]
train_data = data[data['Survived'].notnull()]
X_train = train_data.drop('Survived',axis = 1)
X_test = test_data.drop('Survived',axis = 1)
y_train = train_data['Survived']
cor = pd.read_csv('cheat.csv')
y_cheat = cor['Survived']

In [11]:
clf1 = XGBClassifier(reg_alpha=1e-05, reg_lambda = 0.5,learning_rate =0.1, n_estimators=120, max_depth=4,\
                                                  min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8,\
                                                  objective= 'binary:logistic', nthread=4,     scale_pos_weight=1, seed=27)

xgb2 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)

In [12]:
from sklearn.model_selection import cross_val_score
print(cross_val_score(clf1,X_train,y_train,cv=3).mean())
print(cross_val_score(xgb2,X_train,y_train,cv=3).mean())

0.828282828283


0.811447811448


In [13]:
from sklearn.metrics import accuracy_score
clf1.fit(X_train,y_train)
y_pred = clf1.predict(X_test)
print(accuracy_score(y_cheat,y_pred))

0.767942583732


Try to remove corr
===================

In [131]:
while True:
    df_corr=data.drop(['Survived'],axis=1).corr(method='spearman')
    mask=np.ones(df_corr.columns.size)-np.eye(df_corr.columns.size)
    df_corr=df_corr*mask
    drops=[]
    for col in df_corr.columns.values:
        if np.in1d([col],drops):
            continue
        corr=df_corr.index[abs(df_corr[col])>0.9].values
        drops=np.union1d(drops,corr)
    print("\nDropping",drops.shape[0],"highly correlated features")
    data_nocorr=data.drop(drops,axis=1,inplace=False)
    break



Dropping 113 highly correlated features


In [133]:
train_df=data_nocorr[data_nocorr['Survived'].notnull()]
test_df=data_nocorr[data_nocorr['Survived'].isnull()]
test_df.reset_index(inplace=True)
test_df.drop('index',axis=1,inplace=True)
test_df.drop('Survived',axis=1,inplace=True)
train_df.drop('Survived',axis =1,inplace=True)



C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [110]:
from sklearn.model_selection import cross_val_score
print(cross_val_score(clf1,train_df,y_train,cv=3).mean())
print(cross_val_score(xgb2,train_df,y_train,cv=3).mean())

0.819304152637


0.809203142536


In [135]:
from sklearn.metrics import accuracy_score
xgb2.fit(train_df,y_train)
y_pred = xgb2.predict(test_df)
print(accuracy_score(y_cheat,y_pred))


0.748803827751


In [137]:
from sklearn.ensemble import RandomForestClassifier
rf3 = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=4, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=5, verbose=0,
            warm_start=False)

In [139]:
rf3.fit(train_df,y_train)
y_pred = rf3.predict(test_df)
print(accuracy_score(y_cheat,y_pred))
print(rf3.feature_importances_)

0.772727272727
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.00112227  0.15382214  0.00080862  0.          0.
  0.00988304  0.          0.          0.          0.          0.
  0.00064632  0.          0.          0.          0.          0.00059543
  0.00614748  0.          0.00387736  0.03704086  0.          0.
  0.01253539  0.00328762  0.          0.          0.05452906  0.
  0.00212238  0.00209229  0.00702707  0.01103759  0.          0.00153212
  0.          0.          0.          0.          0.01372497  0.00947235
  0.          0.          0.00429448  0.          0.          0.00876025
  0.          0.          0.00588487  0.          0.          0.          0.
  0.          0.          0.01411209  0.          0.          0.02868499
  0.          0.00145351  0.00142605  0.          0.00249816  0.          0.
  0.          0.00066693  0.00609633  0.          0.          0.          0.0013821
  0.          0.00434597  0.     

In [141]:
rf3_im =  rf3.feature_importances_
# a = np.argpartition(rf3_im,-20)[-20:]
# rf3_im[a]

# im.shape
# X_all = data.drop('Survived',axis=1)
namelist = np.array(train_df.columns.tolist())

train_df_short =train_df.drop(namelist[rf3_im <=1e-5],axis=1)
test_df_short =test_df.drop(namelist[rf3_im <=1e-5],axis=1)

In [142]:
rf3.fit(train_df_short,y_train)
y_pred = rf3.predict(test_df_short)
print(accuracy_score(y_cheat,y_pred))
print(accuracy_score(y_train,rf3.predict(train_df_short)))

0.782296650718
0.874298540965


In [76]:
stack_train= pd.DataFrame([])
stack_test = pd.DataFrame([])
stack_train['df_0.782'] = rf3.predict(train_df_short)
stack_test['df_0.782'] = rf3.predict(test_df_short)

In [78]:
clf1.fit(X_train,y_train)
y_pre_xg = clf1.predict(X_test)
print(accuracy_score(y_pre_xg,y_cheat))

0.767942583732


In [82]:

xg_im =  clf1.feature_importances_
# a = np.argpartition(rf3_im,-20)[-20:]
# rf3_im[a]

# im.shape
# X_all = data.drop('Survived',axis=1)
namelist = np.array(X_train.columns.tolist())

train_df_short_xg =X_train.drop(namelist[xg_im <=1e-5],axis=1)
test_df_short_xg =X_test.drop(namelist[xg_im <=1e-5],axis=1)

In [86]:
clf2 = clf1
clf2.fit(train_df_short_xg,y_train)
y_pre_xg = clf2.predict(test_df_short_xg)
# print(accuracy_score(test_df_short_xg,y_cheat))

In [88]:
stack_train['xg_0.77'] = clf2.predict(train_df_short_xg)
stack_test['xg_0.77'] = clf2.predict(test_df_short_xg)

In [94]:
stack_train[stack_train.iloc[:,0]  != stack_train.iloc[:,1]].shape

(72, 2)

In [144]:
stack_train.to_csv('stack_train.csv')
stack_test.to_csv('stack_test.csv')